In [34]:
import pandas as pd
import logging
import time

In [35]:
player_df = pd.read_csv('../../data/player_stat_2022.csv')
player_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,42200116,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,F,NaN,39.000000:35,...,0.0,5.0,5.0,2.0,0.0,1.0,2.0,3.0,32.0,7.0
1,42200116,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,NaN,39.000000:52,...,2.0,12.0,14.0,7.0,1.0,2.0,0.0,1.0,30.0,11.0
2,42200116,1610612738,BOS,Boston,201143,Al Horford,Al,C,NaN,29.000000:54,...,3.0,9.0,12.0,4.0,2.0,3.0,0.0,4.0,10.0,9.0
3,42200116,1610612738,BOS,Boston,1628401,Derrick White,Derrick,G,NaN,29.000000:46,...,1.0,1.0,2.0,2.0,0.0,1.0,1.0,3.0,7.0,-1.0
4,42200116,1610612738,BOS,Boston,203935,Marcus Smart,Marcus,G,NaN,32.000000:33,...,3.0,1.0,4.0,4.0,1.0,0.0,2.0,0.0,22.0,11.0


In [36]:
player_df.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'NICKNAME', 'START_POSITION', 'COMMENT', 'MIN', 'FGM',
       'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
       'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS',
       'PLUS_MINUS'],
      dtype='object')

In [37]:
game_df = pd.read_csv('../../data/game_2022.csv')
# cleaning, game_df is unique on team_id and game_id level after fixing
game_df = game_df.dropna(subset=['WL'])
game_df.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42022,1610612737,ATL,Atlanta Hawks,42200116,2023-04-27,ATL vs. BOS,L,241,120,...,0.773,12.0,33.0,45.0,28,5.0,7,10,15,-8.0
1,42022,1610612737,ATL,Atlanta Hawks,42200115,2023-04-25,ATL @ BOS,W,242,119,...,1.000,6.0,28.0,34.0,26,5.0,4,8,16,2.0
2,42022,1610612737,ATL,Atlanta Hawks,42200114,2023-04-23,ATL vs. BOS,L,240,121,...,0.875,11.0,31.0,42.0,25,8.0,4,12,24,-8.0
3,42022,1610612737,ATL,Atlanta Hawks,42200113,2023-04-21,ATL vs. BOS,W,240,130,...,0.813,11.0,37.0,48.0,24,5.0,6,18,15,8.0
4,42022,1610612737,ATL,Atlanta Hawks,42200112,2023-04-18,ATL @ BOS,L,241,106,...,0.500,19.0,30.0,49.0,21,10.0,4,15,11,-13.0


In [38]:
player_df.shape

(39273, 29)

In [39]:
game_df.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [40]:
# Merge player_df with game_df to get the date of each game
merged_df = player_df.merge(game_df[['GAME_ID', 'GAME_DATE', 'TEAM_ID',  'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']], on=['GAME_ID', 'TEAM_ID'], how='left', suffixes=(None, '_TEAM'))
# Convert GAME_DATE to datetime
merged_df['GAME_DATE'] = pd.to_datetime(merged_df['GAME_DATE'])
merged_df.shape

(39320, 50)

In [41]:
# Sort the DataFrame by player and date
merged_df.sort_values(['PLAYER_ID', 'GAME_DATE'], inplace=True)

In [42]:
# Define a window size for calculating recent averages
window_sizes = [3,5,10,15,30]  # You can adjust this as needed

# Calculate rolling averages for various statistics
rolling_cols = ['PTS', 'REB', 'AST', 'FGM', 'FG3M', 'FTM']
for col in rolling_cols:
    for window in window_sizes:
        merged_df[f'{col}_avg_{window}'] = merged_df.groupby('PLAYER_ID')[col].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

merged_df.head(20)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,FG3M_avg_3,FG3M_avg_5,FG3M_avg_10,FG3M_avg_15,FG3M_avg_30,FTM_avg_3,FTM_avg_5,FTM_avg_10,FTM_avg_15,FTM_avg_30
24021,12200012,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,15.000000:41,...,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,4.000000,4.000000,4.000000,4.000000
23982,12200022,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:26,...,1.500000,1.500000,1.500000,1.500000,1.500000,4.000000,4.000000,4.000000,4.000000,4.000000
23958,12200028,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,1.500000,1.500000,1.500000,1.500000,1.500000,4.000000,4.000000,4.000000,4.000000,4.000000
18508,12200040,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,3.000000,1.500000,1.500000,1.500000,1.500000,4.000000,4.000000,4.000000,4.000000,4.000000
23904,12200057,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,24.000000:42,...,2.000000,1.666667,1.666667,1.666667,1.666667,5.000000,4.333333,4.333333,4.333333,4.333333
23846,12200070,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:47,...,1.000000,1.666667,1.250000,1.250000,1.250000,3.500000,3.666667,3.750000,3.750000,3.750000
18427,22200002,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,35.000000:12,...,1.666667,1.666667,1.600000,1.600000,1.600000,3.666667,3.666667,3.800000,3.800000,3.800000
22119,22200016,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,36.000000:40,...,1.666667,1.750000,1.666667,1.666667,1.666667,3.333333,3.750000,3.833333,3.833333,3.833333
23832,22200037,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,37.000000:43,...,2.333333,1.800000,1.714286,1.714286,1.714286,4.333333,4.000000,4.000000,4.000000,4.000000
16100,22200064,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,35.000000:19,...,2.000000,1.800000,1.750000,1.750000,1.750000,3.333333,3.200000,3.625000,3.625000,3.625000


In [43]:
merged_df.shape

(39320, 80)

In [44]:
# Merge the dataframes based on game_id and TEAM_ID mismatch
merged_df = merged_df.merge(game_df[['GAME_ID', 'TEAM_ID', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']], on='GAME_ID', how='left', suffixes=(None, '_OPP'))
merged_df = merged_df[~merged_df['TEAM_ID'].eq(merged_df['TEAM_ID_OPP'])]
merged_df.shape
# Now merged_df contains the rows where game_id matches but TEAM_ID is different

(39276, 101)

In [45]:
merged_df.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,FT_PCT_OPP,OREB_OPP,DREB_OPP,REB_OPP,AST_OPP,STL_OPP,BLK_OPP,TOV_OPP,PF_OPP,PLUS_MINUS_OPP
1,12200012,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,15.000000:41,...,0.632,15.0,39.0,54.0,25,14.0,8,7,25,30.0
3,12200022,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:26,...,0.848,16.0,39.0,55.0,21,10.0,5,11,30,4.0
5,12200028,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,0.769,9.0,50.0,59.0,29,11.0,6,15,31,15.0
6,12200040,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,0.706,11.0,34.0,45.0,22,10.0,3,19,22,-3.0
9,12200057,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,24.000000:42,...,0.733,9.0,43.0,52.0,29,10.0,4,12,25,5.0


In [46]:
# Define a window size for calculating recent averages
window_sizes = [5,10,15,30]  # You can adjust this as needed

# Calculate rolling averages of opponent defensive statistics
rolling_cols = ['DREB_OPP', 'STL_OPP', 'BLK_OPP']
for col in rolling_cols:
    for window in window_sizes:
        merged_df[f'{col}_avg_{window}'] = merged_df.groupby('PLAYER_ID')[col].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

merged_df.head(20)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,DREB_OPP_avg_15,DREB_OPP_avg_30,STL_OPP_avg_5,STL_OPP_avg_10,STL_OPP_avg_15,STL_OPP_avg_30,BLK_OPP_avg_5,BLK_OPP_avg_10,BLK_OPP_avg_15,BLK_OPP_avg_30
1,12200012,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,15.000000:41,...,39.000000,39.000000,14.000000,14.000000,14.000000,14.000000,8.000000,8.000000,8.000000,8.000000
3,12200022,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:26,...,39.000000,39.000000,12.000000,12.000000,12.000000,12.000000,6.500000,6.500000,6.500000,6.500000
5,12200028,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,42.666667,42.666667,11.666667,11.666667,11.666667,11.666667,6.333333,6.333333,6.333333,6.333333
6,12200040,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,40.500000,40.500000,11.250000,11.250000,11.250000,11.250000,5.500000,5.500000,5.500000,5.500000
9,12200057,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,24.000000:42,...,41.000000,41.000000,11.000000,11.000000,11.000000,11.000000,5.200000,5.200000,5.200000,5.200000
11,12200070,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:47,...,41.000000,41.000000,10.400000,11.000000,11.000000,11.000000,3.800000,4.500000,4.500000,4.500000
12,22200002,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,35.000000:12,...,40.428571,40.428571,10.600000,11.000000,11.000000,11.000000,3.600000,4.428571,4.428571,4.428571
14,22200016,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,36.000000:40,...,41.375000,41.375000,9.800000,10.500000,10.500000,10.500000,3.800000,4.750000,4.750000,4.750000
17,22200037,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,37.000000:43,...,41.111111,41.111111,9.000000,10.000000,10.000000,10.000000,4.200000,4.777778,4.777778,4.777778
18,22200064,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,35.000000:19,...,41.500000,41.500000,9.400000,10.200000,10.200000,10.200000,4.400000,4.800000,4.800000,4.800000


In [53]:
# Define a window size for calculating recent averages
window_sizes = [5,10,15]  # You can adjust this as needed

# Calculate rolling averages of team performance metrics
team_performance_metrics = ['PTS_TEAM',
    'REB_TEAM',
    'AST_TEAM',
    'STL_TEAM',
    'BLK_TEAM',
    'FG_PCT_TEAM',
    'FG3_PCT_TEAM',
    'FT_PCT_TEAM',
    'TOV',
    'PLUS_MINUS_TEAM']
for col in team_performance_metrics:
    for window in window_sizes:
        merged_df[f'{col}_avg_{window}'] = merged_df.groupby('PLAYER_ID')[col].rolling(window=window, min_periods=1).mean().reset_index(level=0, drop=True)

merged_df.head(20)

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,FG3_PCT_TEAM_avg_15,FT_PCT_TEAM_avg_5,FT_PCT_TEAM_avg_10,FT_PCT_TEAM_avg_15,TOV_avg_5,TOV_avg_10,TOV_avg_15,PLUS_MINUS_TEAM_avg_5,PLUS_MINUS_TEAM_avg_10,PLUS_MINUS_TEAM_avg_15
1,12200012,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,15.000000:41,...,0.233000,0.62500,0.625000,0.625000,20.00,20.000000,20.000000,-30.000000,-30.000000,-30.000000
3,12200022,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:26,...,0.275000,0.67600,0.676000,0.676000,18.00,18.000000,18.000000,-17.000000,-17.000000,-17.000000
5,12200028,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,0.258000,0.69500,0.695000,0.695000,18.00,18.000000,18.000000,-16.333333,-16.333333,-16.333333
6,12200040,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,NaN,DNP - Coach's Decision,NaN,...,0.302500,0.75725,0.757250,0.757250,17.75,17.750000,17.750000,-11.500000,-11.500000,-11.500000
9,12200057,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,24.000000:42,...,0.293200,0.76260,0.762600,0.762600,18.20,18.200000,18.200000,-10.200000,-10.200000,-10.200000
11,12200070,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,17.000000:47,...,0.284667,0.76660,0.743000,0.743000,18.60,18.833333,18.833333,-12.000000,-15.000000,-15.000000
12,22200002,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,35.000000:12,...,0.279714,0.77320,0.745429,0.745429,19.60,19.142857,19.142857,-14.000000,-14.857143,-14.857143
14,22200016,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,36.000000:40,...,0.269750,0.79580,0.758000,0.758000,18.00,18.000000,18.000000,-12.200000,-13.750000,-13.750000
17,22200037,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,37.000000:43,...,0.260000,0.76700,0.762667,0.762667,17.20,17.444444,17.444444,-13.200000,-12.444444,-12.444444
18,22200064,1610612747,LAL,Los Angeles,2544,LeBron James,LeBron,F,NaN,35.000000:19,...,0.260700,0.74780,0.755200,0.755200,16.00,17.100000,17.100000,-14.400000,-12.300000,-12.300000


In [54]:
merged_df.START_POSITION.unique()

array(['F', nan, 'C', 'G'], dtype=object)

In [55]:
merged_df.COMMENT.unique()

array([nan, "DNP - Coach's Decision", 'DND - Injury/Illness',
       'DNP - Injury/Illness', 'NWT - Personal', 'NWT - Not With Team',
       'DND-Return to Competition Reconditioning', 'DNP - Rest',
       'DND - Rest', 'NWT - Injury/Illness', 'DNP - League Suspension',
       'NWT - League Suspension', "DND - Coach's Decision",
       'NWT-Return to Competition Reconditioning', 'NWT_TEAM_SUSPENSION',
       'DND_LEAGUE_SUSPENSION', 'DNP - Personal', 'DND - Personal',
       'NWT - Trade Pending', 'DNP - Concussion Protocol',
       'DND - Concussion Protocol', 'NWT - Health and Safety Protocols',
       'DND - Health and Safety Protocols'], dtype=object)

In [56]:
merged_df.to_csv('../../data/player_stat_team_stat_aggregate.csv', index = False)